In [1]:
import LowLevelFEM as FEM
using LowLevelFEM

gmsh.initialize()

In [2]:
gmsh.open("thermalStress.geo")

Info    : Reading 'thermalStress.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00064165s, CPU 0.000642s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 0.000781526s, CPU 0.000781s)
Info    : 1111 nodes 1224 elements
Info    : Done reading 'thermalStress.geo'


In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:PlaneHeatConduction, bandwidth=:none)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 22)


LowLevelFEM.Problem("bending2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5)], 1.0, 1111)

In [4]:
K = FEM.heatConductionMatrix(problem)

1111×1111 SparseArrays.SparseMatrixCSC{Float64, Int64} with 9331 stored entries:
⎡⠻⣦⡀⠈⠁⠀⠀⠈⠉⠓⠒⠒⠒⠲⠤⠤⠤⠤⣄⣀⣀⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⎤
⎢⡀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠙⠓⠒⠒⠒⠶⠤⠤⠤⢤⣄⣀⣀⣀⎥
⎢⠁⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣀⣀⣀⣤⠤⠤⠤⠴⠖⠒⠒⠒⠛⠉⠉⎥
⎢⡀⠀⠀⠀⠀⠈⠻⣦⣀⣀⣤⠤⠤⠤⠴⠖⠒⠒⠒⠛⠉⠉⠉⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢧⠀⠀⠀⠀⠀⠀⢸⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢸⠀⠀⠀⠀⠀⠀⡟⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢸⡀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⡇⠀⠀⠀⠀⢰⠇⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⡇⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⢹⠀⠀⠀⠀⣼⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⢸⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠘⡇⠀⠀⢀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⡇⠀⠀⢸⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⢷⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⢸⠀⠀⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⢸⡄⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⡇⢰⠇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⣇⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⢹⣼⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⎥
⎣⡄⠀⠀⢸⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⎦

In [5]:
left = FEM.temperatureConstraint("left", T=100)
right = FEM.temperatureConstraint("right", T=100)
flux = FEM.heatFlux("left", qn=0)
#conv = FEM.heatConvection("outer", h=0.01, Tₐ=-10)

("left", 0, 0, 0)

In [6]:
q = FEM.loadVector(problem, [flux]);

In [7]:
FEM.applyBoundaryConditions!(problem, K, q, [left, right])

In [8]:
T = FEM.solveTemperature(K, q);

In [9]:
Q = FEM.solveHeatFlux(problem, T)

LowLevelFEM.VectorField([[3.197442310920451e-12; 0.0; … ; 0.0; 0.0;;], [2.5579538487363607e-12; 0.0; … ; 0.0; 0.0;;], [3.197442310920451e-12; 0.0; … ; 0.0; 0.0;;], [3.836930773104541e-12; 0.0; … ; 0.0; 0.0;;], [4.3964831775156215e-12; 0.0; … ; 0.0; 0.0;;], [5.115907697472721e-12; 0.0; … ; 0.0; 0.0;;], [4.476419235288631e-12; 0.0; … ; 0.0; 0.0;;], [5.115907697472721e-12; 0.0; … ; 0.0; 0.0;;], [3.836930773104541e-12; 0.0; … ; 0.0; 0.0;;], [4.476419235288631e-12; 0.0; … ; 0.0; 0.0;;]  …  [-3.836930773104541e-12; 1.2789769243681803e-12; … ; 1.2789769243681803e-12; 0.0;;], [-5.115907697472721e-12; 0.0; … ; 0.0; 0.0;;], [-5.115907697472721e-12; -1.2789769243681803e-12; … ; -1.2789769243681803e-12; 0.0;;], [-3.7569947153315305e-12; -6.394884621840902e-13; … ; -6.394884621840902e-13; 0.0;;], [-3.197442310920451e-12; -6.394884621840902e-13; … ; -6.394884621840902e-13; 0.0;;], [-2.5579538487363607e-12; -6.394884621840902e-13; … ; -6.394884621840902e-13; 0.0;;], [-1.9184653865522705e-12; -6.39488

In [10]:
T1 = FEM.showDoFResults(problem, T, :T, name="T", visible=true)

1

In [11]:
qq = FEM.showHeatFluxResults(problem, Q, :q, visible=false, smooth=false)

2

In [12]:
FEM.plotOnPath(problem, "path", T1, name="T", visible=false);

In [13]:
problem2 = FEM.Problem([mat], type=:PlaneStress, bandwidth=:none)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 22)
Info    : Renumbering nodal model data (1 step)


LowLevelFEM.Problem("bending2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5)], 1.0, 1111)

In [14]:
K2 = FEM.stiffnessMatrix(problem2)

2222×2222 SparseArrays.SparseMatrixCSC{Float64, Int64} with 37026 stored entries:
⎡⠻⣦⡀⠈⠁⠀⠀⠈⠙⠓⠒⠒⠒⠶⠤⠤⠤⢤⣄⣀⣀⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⎤
⎢⡀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠙⠓⠒⠒⠒⠶⠤⠤⠤⢤⣄⣀⣀⣀⎥
⎢⠁⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣀⣀⣀⣤⠤⠤⠤⠴⠖⠒⠒⠒⠛⠉⠉⎥
⎢⡀⠀⠀⠀⠀⠈⠻⣦⣀⣀⣤⠤⠤⠤⠴⠖⠒⠒⠒⠛⠉⠉⠉⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢷⠀⠀⠀⠀⠀⠀⢸⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢸⠀⠀⠀⠀⠀⠀⡟⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢸⡄⠀⠀⠀⠀⠀⡇⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⡇⠀⠀⠀⠀⢰⠇⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⣇⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⢹⠀⠀⠀⠀⣼⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⢸⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠘⡇⠀⠀⢀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⡇⠀⠀⢸⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⢷⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⢸⠀⠀⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⢸⡄⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⡇⢰⠇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⣇⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⢹⣼⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⎥
⎣⡄⠀⠀⢸⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⎦

In [15]:
left2 = FEM.displacementConstraint("left", ux=0)
right2 = FEM.displacementConstraint("right", ux=0)
bottom2 = FEM.displacementConstraint("bottom", uy=0)
load = FEM.load("left", fx=0);

In [16]:
f = FEM.loadVector(problem2, [load]);

In [17]:
fT = FEM.thermalLoadVector(problem2, T);

In [18]:
f2 = f + fT;

In [19]:
FEM.applyBoundaryConditions!(problem2, K2, f2, [left2, right2, bottom2])
#FEM.applyBoundaryConditions!(problem2, K2, f2, [left2, bottom2])

In [20]:
q2 = FEM.solveDisplacement(K2, f2);

In [21]:
E = FEM.solveStrain(problem2, q2)

LowLevelFEM.TensorField([[1.7248408772231114e-17; 0.0; … ; 0.0; -0.0006685714285714326;;], [1.749640516969843e-17; -4.336808689942018e-19; … ; 0.0; -0.0006685714285714323;;], [1.825158741879874e-17; 8.673617379884035e-19; … ; 0.0; -0.0006685714285714322;;], [1.819431615985705e-17; 2.6020852139652106e-18; … ; 0.0; -0.0006685714285714326;;], [1.8417956787545343e-17; 8.618106228652541e-19; … ; 0.0; -0.0006685714285714326;;], [1.9333526038919352e-17; 0.0; … ; 0.0; -0.0006685714285714344;;], [1.8940233499373492e-17; -5.204170427930421e-18; … ; 0.0; -0.0006685714285714337;;], [1.994191222135464e-17; -3.469446951953614e-18; … ; 0.0; -0.000668571428571437;;], [2.2248030086435204e-17; -1.0408340855860843e-17; … ; 0.0; -0.0006685714285714344;;], [2.4844558709962926e-17; -5.204170427930421e-18; … ; 0.0; -0.000668571428571438;;]  …  [1.6471155452677077e-17; -1.5622297018205188e-19; … ; 0.0; -0.0006685714285714332;;], [1.662737842285913e-17; 3.459155328770308e-19; … ; 0.0; -0.0006685714285714328;;]

In [22]:
S = FEM.solveStress(problem2, q2, T=T)

LowLevelFEM.TensorField([[-239.99999999999673; 0.0; … ; 0.0; 0.0;;], [-239.99999999999676; -3.336006684570783e-14; … ; 0.0; 0.0;;], [-239.99999999999662; 6.672013369141566e-14; … ; 0.0; 0.0;;], [-239.9999999999966; 2.0016040107424696e-13; … ; 0.0; 0.0;;], [-239.9999999999966; 6.629312483578878e-14; … ; 0.0; 0.0;;], [-239.99999999999608; 0.0; … ; 0.0; 0.0;;], [-239.99999999999636; -4.003208021484939e-13; … ; 0.0; 0.0;;], [-239.9999999999958; -2.6688053476566263e-13; … ; 0.0; 0.0;;], [-239.99999999999585; -8.006416042969878e-13; … ; 0.0; 0.0;;], [-239.99999999999494; -4.003208021484939e-13; … ; 0.0; 0.0;;]  …  [-239.99999999999648; -1.2017151552465529e-14; … ; 0.0; 0.0;;], [-239.99999999999636; 2.6608887144386986e-14; … ; 0.0; 0.0;;], [-239.99999999999642; 5.630469660797252e-14; … ; 0.0; 0.0;;], [-239.99999999999645; 2.0430134910506332e-13; … ; 0.0; 0.0;;], [-239.99999999999662; 2.497208925086009e-13; … ; 0.0; 0.0;;], [-239.99999999999687; 3.7518491844877614e-13; … ; 0.0; 0.0;;], [-239.9

In [23]:
u = FEM.showDoFResults(problem2, q2, :uvec, visible=false)
ux = FEM.showDoFResults(problem2, q2, :ux, visible=false)
uy = FEM.showDoFResults(problem2, q2, :uy, visible=false)

6

In [24]:
ex = FEM.showStrainResults(problem2, E, :ex, visible=false)
ey = FEM.showStrainResults(problem2, E, :ey, visible=false)
exy = FEM.showStrainResults(problem2, E, :exy, visible=false)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


9

In [25]:
sx = FEM.showStressResults(problem2, S, :sx, visible=false)
sy = FEM.showStressResults(problem2, S, :sy, visible=false)
sxy = FEM.showStressResults(problem2, S, :sxy, visible=false)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


12

In [26]:
ΔT = 100
α = problem.material[1].α
ν = problem.material[1].ν
E = problem.material[1].E
l₀ = 100
Δl = l₀ * α * ΔT
ε₀ = α * ΔT
ε₁ = -Δl / l₀
ε₁ₖ = -ν * ε₁
σₓ = E * ε₁
p = E * α / 3 / (1 - 2ν) * ΔT
display("Δl = $Δl")
display("ε₀ = $ε₀")
display("ε₁ = $ε₁")
display("ε₁ₖ = $ε₁ₖ")
display("ε₁ₖ+ε₀ = $(ε₁ₖ+ε₀)")
display("σₓ = $σₓ")
display("p = $p")

"Δl = 0.12000000000000001"

"ε₀ = 0.0012000000000000001"

"ε₁ = -0.0012000000000000001"

"ε₁ₖ = 0.00036"

"ε₁ₖ+ε₀ = 0.0015600000000000002"

"σₓ = -240.00000000000003"

"p = 199.99999999999997"

In [27]:
ε₁ = Δl / l₀

0.0012000000000000001

In [28]:
FF = FEM.showDoFResults(problem2, fT, :uvec, name="fT", visible=false)

13

In [29]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [30]:
gmsh.finalize()